# Start LangGraph


In [1]:
# !pip install langchain==0.2.0
# !pip install langchain-openai==0.1.7
# !pip install langchain-community==0.2.0
# !pip install langgraph==0.1.1
# !pip install langchain-chroma==0.1.1        

In [2]:
# pip install sqlalchemy

In [ ]:
import os
import re
import json
import yaml
import pandas as pd
import psycopg2
from openai import OpenAI
from dotenv import load_dotenv

### Setup Environment Variables

In [ ]:
# use file .env to get the API key
from dotenv import load_dotenv
load_dotenv()

import os

OPENAI_KEY = os.getenv("OPENAI_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")

os.environ["OPENAI_API_KEY"] = OPENAI_KEY
os.environ["TAVILY_API_KEY"] = TAVILY_API_KEY


## Build a Vector Database for Wikipedia Data

### Open AI Embedding Models

In [ ]:
from langchain_openai import OpenAIEmbeddings
openai_embed_model = OpenAIEmbeddings(model='text-embedding-3-small')

### Load and Chunk Documents

In [ ]:
import gzip
import json
from langchain.docstore.document import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
wikipedia_filepath = './data/simplewiki-2020-11-01.jsonl.gz'
docs = []
with gzip.open(wikipedia_filepath, 'rt', encoding='utf8') as fIn:
    for line in fIn:
        data = json.loads(line.strip())
        # Add documents
        docs.append({
            'metadata': {
                'title': data.get('title'),
                'article_id': data.get('id')
            },
            'data': ' '.join(data.get('paragraphs')[0:3])  # restrict data to first 3 paragraphs to run later modules faster
            })
docs = [doc for doc in docs for x in ['india'] if x in doc['data'].lower().split()]
docs = [Document(page_content=doc['data'], metadata=doc['metadata']) for doc in docs]
splitter = RecursiveCharacterTextSplitter(chunk_size=2000, chunk_overlap=300)
chunked_docs = splitter.split_documents(docs)
chunked_docs[:3]

[Document(metadata={'title': 'Basil', 'article_id': '73985'}, page_content='Basil ("Ocimum basilicum") ( or ) is a plant of the Family Lamiaceae. It is also known as Sweet Basil or Tulsi. It is a tender low-growing herb that is grown as a perennial in warm, tropical climates. Basil is originally native to India and other tropical regions of Asia. It has been cultivated there for more than 5,000 years. It is prominently featured in many cuisines throughout the world. Some of them are Italian, Thai, Vietnamese and Laotian cuisines. It grows to between 30–60\xa0cm tall. It has light green, silky leaves 3–5\xa0cm long and 1–3\xa0cm broad. The leaves are opposite each other. The flowers are quite big. They are white in color and arranged as a spike. The plant tastes somewhat like anise, with a strong, pungent, sweet smell. Basil is very sensitive to cold. It is best grown in hot, dry conditions. While most common varieties are treated as annuals, some are perennial, including African Blue a

### Create a Vector DB and Persist on the Disk

In [ ]:
from langchain_chroma import Chroma
chroma_db = Chroma.from_documents(documents=chunked_docs, collection_name='rag_wikipedia_db',embedding=openai_embed_model, collection_metadata={"hnsw:space": "cosine"}, persist_directory="./wikipedia_db")

### Setup a Vector Database Retriever

In [9]:
similarity_threshold_retriever = chroma_db.as_retriever(search_type="similarity_score_threshold", search_kwargs={"k": 3, "score_threshold": 0.3})
# We can then test if our retriever is working on some sample queries.
query = "what is the capital of India?"
top3_docs = similarity_threshold_retriever.invoke(query)
top3_docs

[Document(id='44b3bb5a-56e2-4249-8fac-fa774d3bb59c', metadata={'article_id': '5117', 'title': 'New Delhi'}, page_content='New Delhi () is the capital of India and a union territory of the megacity of Delhi. It has a very old history and is home to several monuments where the city is expensive to live in. In traditional Indian geography it falls under the North Indian zone. The city has an area of about 42.7\xa0km. New Delhi has a population of about 9.4 Million people.'),
 Document(id='eb56ab47-4627-4cf9-80bd-7bde427ede7f', metadata={'article_id': '5117', 'title': 'New Delhi'}, page_content='New Delhi () is the capital of India and a union territory of the megacity of Delhi. It has a very old history and is home to several monuments where the city is expensive to live in. In traditional Indian geography it falls under the North Indian zone. The city has an area of about 42.7\xa0km. New Delhi has a population of about 9.4 Million people.'),
 Document(id='8cb8776c-ce5a-46fd-bd4d-64e97a6b

In [10]:
query = "what is langgraph?"
top3_docs = similarity_threshold_retriever.invoke(query)
top3_docs

No relevant docs were retrieved using the relevance score threshold 0.3


[]

## Create a Query Retrieval Grader

In [11]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI
from langchain_core.documents import Document

# Tạo LLM với cấu hình Pydantic V2
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

# Parser
parser = StrOutputParser()

# Prompt system
SYS_PROMPT = """
You are an expert grader assessing relevance of a retrieved document to a user question.
Answer only 'yes' or 'no' depending on whether the document is relevant to the question.
"""

# Tạo prompt template
grade_prompt = ChatPromptTemplate.from_messages([
    ("system", SYS_PROMPT),
    ("human", "Retrieved document:\n{document}\n\nUser question:\n{question}")
])

# Tạo chain xử lý
doc_grader = grade_prompt | llm | parser

In [12]:
query = "what is the capital of India?"
top3_docs = similarity_threshold_retriever.invoke(query)
for doc in top3_docs:
	print(doc.page_content)
	print('GRADE:', doc_grader.invoke({
		"question": query,
		"document": doc.page_content
	}))
	print()

New Delhi () is the capital of India and a union territory of the megacity of Delhi. It has a very old history and is home to several monuments where the city is expensive to live in. In traditional Indian geography it falls under the North Indian zone. The city has an area of about 42.7 km. New Delhi has a population of about 9.4 Million people.
GRADE: yes

New Delhi () is the capital of India and a union territory of the megacity of Delhi. It has a very old history and is home to several monuments where the city is expensive to live in. In traditional Indian geography it falls under the North Indian zone. The city has an area of about 42.7 km. New Delhi has a population of about 9.4 Million people.
GRADE: yes

New Delhi () is the capital of India and a union territory of the megacity of Delhi. It has a very old history and is home to several monuments where the city is expensive to live in. In traditional Indian geography it falls under the North Indian zone. The city has an area of 

## Build a QA RAG Chain

In [13]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter

# Create RAG prompt for response generation
prompt = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If no context is present or if you don't know the answer, just say that you don't know the answer.
Do not make up the answer unless it is there in the provided context.
Give a detailed answer and to the point answer with regard to the question.
Question:
{question}
Context:
{context}
Answer:
"""
prompt_template = ChatPromptTemplate.from_template(prompt)

# Initialize connection with gpt-4.1-mini
chatgpt = ChatOpenAI(model_name='gpt-4.1-mini', temperature=0)

# Used for separating context docs with new lines
def format_docs(docs):
	return "\n\n".join(doc.page_content for doc in docs)

# Create QA RAG chain
qa_rag_chain = (
	{
		"context": (itemgetter('context')
					|
					RunnableLambda(format_docs)),
		"question": itemgetter('question')
	}
	|
	prompt_template
	|
	chatgpt
	|
	StrOutputParser()
)

In [14]:
query = "what is the capital of India?"
top3_docs = similarity_threshold_retriever.invoke(query)
result = qa_rag_chain.invoke({"context": top3_docs, "question": query})
print(result)

The capital of India is New Delhi. It is a union territory within the megacity of Delhi and is located in the North Indian zone according to traditional Indian geography. New Delhi has a rich history, is home to several monuments, and covers an area of about 42.7 square kilometers. The city has a population of approximately 9.4 million people.


In [15]:
query = "who won the champions league in 2024?"
top3_docs = similarity_threshold_retriever.invoke(query)
result = qa_rag_chain.invoke({"context": top3_docs, "question": query})
print(result)

The provided context does not contain any information about the winner of the Champions League in 2024. Therefore, I do not know the answer to who won the Champions League in 2024 based on the given information.


## Create a Query Rephraser

In [16]:
# LLM for question rewriting
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)
# Prompt template for rewriting
SYS_PROMPT = """Act as a question re-writer and perform the following task:
				 - Convert the following input question to a better version that is optimized for web search.
				 - When re-writing, look at the input question and try to reason about the underlying semantic intent / meaning.
			 """
re_write_prompt = ChatPromptTemplate.from_messages(
	[
		("system", SYS_PROMPT),
		("human", """Here is the initial question:
					 {question}
					 Formulate an improved question.
				  """)
	]
)

# Create rephraser chain
question_rewriter = (re_write_prompt|llm|StrOutputParser())

In [17]:
query = "who won the champions league in 2024?"
question_rewriter.invoke({"question": query})

'Who was the winner of the 2024 UEFA Champions League?'

# Load Web Search Tool

In [18]:
# %pip install langchain-core

In [19]:
# Load Web Search Tool
from langchain_community.tools.tavily_search import TavilySearchResults
tv_search = TavilySearchResults(max_results=3, search_depth='advanced', max_tokens=10000)

In [ ]:
load_dotenv()

DB_CONFIG = {
    "dbname": os.getenv("DBNAME") or os.getenv("POSTGRES_DB"),
    "user": os.getenv("DBUSER") or os.getenv("POSTGRES_USER"),
    "password": os.getenv("DBPASSWORD") or os.getenv("POSTGRES_PASSWORD"),
    "host": os.getenv("DBHOST") or "localhost",
    "port": int(os.getenv("DBPORT", 5432)),
    "sslmode": os.getenv("SSL_MODE", "require")
}


print(f"DB Config: {DB_CONFIG}")

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

def connect_to_database():
    try:
        conn = psycopg2.connect(**DB_CONFIG)
        print("Kết nối thành công đến PostgreSQL.")
        return conn
    except Exception as e:
        print(f"Lỗi kết nối cơ sở dữ liệu: {e}")
        return None

def get_schema_and_samples(conn):
    try:
        cursor = conn.cursor()
        cursor.execute("SELECT table_name FROM information_schema.tables WHERE table_schema='public';")
        tables = cursor.fetchall()
        schema_info = {}
        for (table,) in tables:
            cursor.execute(f"SELECT column_name, data_type FROM information_schema.columns WHERE table_name = '{table}';")
            schema_info[table] = [{"column_name": col, "data_type": dtype} for col, dtype in cursor.fetchall()]
            cursor.execute(f"SELECT * FROM {table} LIMIT 3;")
            sample_rows = cursor.fetchall()
            colnames = [desc[0] for desc in cursor.description]
            schema_info[f"{table}_samples"] = [dict(zip(colnames, row)) for row in sample_rows]
        cursor.close()
        return schema_info
    except Exception as e:
        return {"error": str(e)}

def load_metadata():
    try:
        with open("metadata.yml", "r") as file:
            return yaml.safe_load(file)
    except FileNotFoundError:
        return {}

def generate_sql_query(user_question, schema_info=None):
    prompt = f"""
You are a PostgreSQL expert working with a single table called `stocks`.

This table contains daily stock information with the following columns:

- date: Date of the record (format: YYYY-MM-DD)
- price: The closing price of the stock on that date (FLOAT)
- open: The opening price of the stock on that date (FLOAT)
- high: The highest price of the stock on that date (FLOAT)
- low: The lowest price of the stock on that date (FLOAT)
- volume: Trading volume (format: float + 'M' suffix for millions)
- change_percent: Daily percentage change in stock price (can be positive or negative)

Assume this table is already in a PostgreSQL database as `stocks`.

User Question:
{user_question}

Write a correct and optimized PostgreSQL query to answer this question. 
Do not explain or wrap the query in markdown. Just return raw SQL.
    """

    try:
        response = client.chat.completions.create(
            model="gpt-4",
            messages=[{"role": "user", "content": prompt}]
        )
        sql_query = response.choices[0].message.content
        sql_query = re.sub(r'^```sql', '', sql_query).strip('` \n')
        return sql_query
    except Exception as e:
        print(f"Lỗi sinh SQL: {e}")
        return None

def execute_sql_query(conn, query):
    try:
        df = pd.read_sql(query, conn)
        return df
    except Exception as e:
        print(f"Lỗi thực thi SQL: {e}")
        return None
    
def run_chat(question: str):
    conn = connect_to_database()
    if conn is None:
        return

    schema_info = get_schema_and_samples(conn)
    sql_query = generate_sql_query(question, schema_info)

    if not sql_query:
        print("Không thể sinh truy vấn SQL.")
        return

    print(f"\nSQL sinh ra:\n{sql_query}")
    results = execute_sql_query(conn, sql_query)

    if results is None:
        print("Truy vấn lỗi.")
        return

    print("\nDữ liệu:")
    print(results.head(5))

    conn.close()

DB Config: {'dbname': 'postgres', 'user': 'postgres.gmzvpcfwyvluwfwiyqsy', 'password': '6orAlooo94sTKvpY', 'host': 'aws-0-ap-southeast-1.pooler.supabase.com', 'port': 5432, 'sslmode': 'require'}


## Build Agentic RAG components

#### Graph State

In [ ]:
from typing import List
from typing_extensions import TypedDict
class GraphState(TypedDict):
    question: str
    generation: str
    web_search_needed: str
    documents: List[str]
documents: List[str]

#### Retrieve function for retrieval from Vector DB

In [ ]:
def retrieve(state):
	print("---RETRIEVAL FROM VECTOR DB---")
	question = state["question"]
	# Retrieval
	documents = similarity_threshold_retriever.invoke(question)
	return {"documents": documents, "question": question}

#### Grade documents

In [ ]:
def grade_documents(state):
	print("---CHECK DOCUMENT RELEVANCE TO QUESTION---")
	question = state["question"]
	documents = state["documents"]

	filtered_docs = []
	web_search_needed = "No"
	use_sql = "No"

	if documents:
		for d in documents:
			score = doc_grader.invoke(
				{"question": question, "document": d.page_content}
			)
			grade = score.strip().lower()

			if grade == "yes":
				print("---GRADE: DOCUMENT RELEVANT---")
				filtered_docs.append(d)
			else:
				# Phân loại theo nội dung
				if "database" in question.lower() or "truy vấn" in question.lower() or "from table" in question.lower():
					use_sql = "Yes"
					print("---GRADE: DOCUMENT NOT RELEVANT, SUGGEST SQL---")
				else:
					web_search_needed = "Yes"
					print("---GRADE: DOCUMENT NOT RELEVANT, SUGGEST WEB SEARCH---")
	else:
		# Kiểm tra nội dung câu hỏi → có chứa từ khóa SQL
		if "database" in question.lower() or "truy vấn" in question.lower() or "from table" in question.lower():
			use_sql = "Yes"
			print("---NO DOCS, BUT QUESTION SUGGESTS SQL---")
		else:
			web_search_needed = "Yes"

	return {
		"documents": filtered_docs,
		"question": question,
		"web_search_needed": web_search_needed,
		"use_sql": use_sql
	}

#### Rewrite query

In [24]:
def rewrite_query(state):
    """
    Rewrite the query to produce a better question.
    Args:
    state (dict): The current graph state
    Returns:
    state (dict): Updates question key with a re-phrased or re-written question
    """
    print("---REWRITE QUERY---")
    question = state["question"]
    documents = state["documents"]
    # Re-write question
    better_question = question_rewriter.invoke({"question": question})
    return {"documents": documents, "question": better_question}

#### Web Search

In [25]:
from langchain.schema import Document
def web_search(state):
    """
    Web search based on the re-written question.
    Args:
    state (dict): The current graph state
    Returns:
    state (dict): Updates documents key with appended web results
    """
    print("---WEB SEARCH---")
    question = state["question"]
    documents = state["documents"]

    # Web search
    docs = tv_search.invoke(question)
    
    # Gỡ lỗi: kiểm tra cấu trúc trả về
    print("DOCS TYPE:", type(docs))
    print("FIRST ELEMENT TYPE:", type(docs[0]) if docs else "EMPTY")

    # Nếu docs là list of strings
    if isinstance(docs[0], str):
        web_content = "\n\n".join(docs)
    # Nếu docs là list of dicts
    elif isinstance(docs[0], dict) and "content" in docs[0]:
        web_content = "\n\n".join([d["content"] for d in docs])
    # Nếu docs là list of Document
    elif isinstance(docs[0], Document):
        web_content = "\n\n".join([d.page_content for d in docs])
    else:
        raise TypeError("Unsupported doc format")

    web_results = Document(page_content=web_content)
    documents.append(web_results)

    return {"documents": documents, "question": question}

#### Use SQL

In [59]:
pip install tabulate


Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'd:\HK2_2024_2025\Data Platform\Thuc_hanh\CK\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [68]:
from finance_metrics import load_formulas, identify_metric, get_required_fields, compute_metric
from plot_metric import plot_metric
from langchain.schema import Document
import pandas as pd


def query_sql(state):
    print("---EXECUTE SQL QUERY OR METRIC COMPUTATION---")
    question = state["question"]

    # Kết nối đến cơ sở dữ liệu
    conn = connect_to_database()
    if conn is None:
        raise ValueError("Không thể kết nối cơ sở dữ liệu.")

    schema_info = get_schema_and_samples(conn)
    metadata = load_formulas()
    category, metric_name = identify_metric(question, metadata)

    # Nếu là câu hỏi về chỉ số tài chính
    if metric_name:
        print(f"Phát hiện truy vấn liên quan đến chỉ số: {metric_name}")
        required_fields = get_required_fields(category, metric_name, metadata)

        # TA metrics - cần vẽ biểu đồ
        if metric_name in ["RSI", "MACD", "MA", "BollingerBands"]:
            df = pd.read_sql("SELECT date, price FROM stocks ORDER BY date ASC LIMIT 100", conn)
            conn.close()

            image_base64 = plot_metric(metric_name, df)
            result_doc = Document(
                page_content=f"Biểu đồ {metric_name}:",
                metadata={"image_base64": image_base64}
            )

            return {
                "documents": state["documents"] + [result_doc],
                "question": question
            }

        # FA metrics - tính toán dựa trên 1 dòng dữ liệu
        else:
            field_clause = ", ".join(required_fields)
            query = f"SELECT {field_clause} FROM stocks LIMIT 1;"
            df = pd.read_sql(query, conn)
            conn.close()

            if df.empty:
                result_str = f"Không tìm thấy đủ dữ liệu để tính {metric_name}."
            else:
                row = df.iloc[0].to_dict()
                result = compute_metric(metric_name, row)
                result_str = f"{metric_name} = {result}"

            return {
                "documents": state["documents"] + [Document(page_content=result_str)],
                "question": question
            }

    # Truy vấn SQL thông thường
    sql_query = generate_sql_query(question, schema_info)
    if not sql_query:
        raise ValueError("Không thể sinh truy vấn SQL từ câu hỏi.")

    results = execute_sql_query(conn, sql_query)
    conn.close()

    if results is None or results.empty:
        content = "Không có kết quả từ truy vấn SQL."
    else:
        content = results.to_markdown(index=False)  # hoặc `.to_string(index=False)` nếu không muốn bảng Markdown

    # Gán vào document để các bước sau có thể sinh câu trả lời
    doc = Document(page_content=content)

    # Cập nhật state
    return {
        "documents": state["documents"] + [doc],
        "question": question
    }

ModuleNotFoundError: No module named 'matplotlib'

#### Generate Answer

In [71]:
def generate_answer(state):
    print("---GENERATE ANSWER---")
    question = state["question"]
    documents = state["documents"]
    # RAG generation
    generation = qa_rag_chain.invoke({"context": documents, "question": question})
    return {"documents": documents, "question": question,"generation": generation}

#### Decide to Generate

In [72]:
def decide_to_generate(state):
    print("---ASSESS GRADED DOCUMENTS---")
    web_search_needed = state.get("web_search_needed", "No")
    use_sql = state.get("use_sql", "No")

    if use_sql == "Yes":
        print("---DECISION: QUERY SQL DATABASE---")
        return "query_sql"  # Bạn cần tạo node mới: `query_sql`

    elif web_search_needed == "Yes":
        print("---DECISION: WEB SEARCH NEEDED, REWRITE QUERY---")
        return "rewrite_query"

    else:
        print("---DECISION: DOCUMENTS ARE RELEVANT, GENERATE ANSWER---")
        return "generate_answer"


# Build the Agent Graph with LangGraph

In [73]:
from langgraph.graph import END, StateGraph
agentic_rag = StateGraph(GraphState)
# Define the nodes
agentic_rag.add_node("retrieve", retrieve) # retrieve
agentic_rag.add_node("grade_documents", grade_documents) # grade documents
agentic_rag.add_node("rewrite_query", rewrite_query) # transform_query
agentic_rag.add_node("web_search", web_search) # web search
agentic_rag.add_node("query_sql", query_sql) 
agentic_rag.add_node("generate_answer", generate_answer) # generate answer
# Build graph
agentic_rag.set_entry_point("retrieve")
agentic_rag.add_edge("retrieve", "grade_documents")
# Quyết định nhánh tiếp theo sau khi chấm điểm tài liệu
agentic_rag.add_conditional_edges(
    "grade_documents",
    decide_to_generate,
    {
        "rewrite_query": "rewrite_query",
        "generate_answer": "generate_answer",
        "query_sql": "query_sql"
    }
)
agentic_rag.add_edge("rewrite_query", "web_search")
agentic_rag.add_edge("web_search", "generate_answer")
agentic_rag.add_edge("query_sql", "generate_answer")
agentic_rag.add_edge("generate_answer", END)
# Compile
agentic_rag = agentic_rag.compile()

In [74]:
# from IPython.display import Image, display, Markdown
# display(Image(agentic_rag.get_graph().draw_mermaid_png()))

# Test Agentic RAG System

In [75]:
query = "Tính EPS và P/E của công ty Apple trong quý 1 năm 2024"
response = agentic_rag.invoke({"question": query})
display(Markdown(response['generation']))

---RETRIEVAL FROM VECTOR DB---


No relevant docs were retrieved using the relevance score threshold 0.3


---CHECK DOCUMENT RELEVANCE TO QUESTION---
---ASSESS GRADED DOCUMENTS---
---DECISION: WEB SEARCH NEEDED, REWRITE QUERY---
---REWRITE QUERY---
---WEB SEARCH---
DOCS TYPE: <class 'list'>
FIRST ELEMENT TYPE: <class 'dict'>
---GENERATE ANSWER---


For the first quarter of fiscal 2024 (ended December 30, 2023), Apple Inc. reported an earnings per share (EPS) of $2.18. This represented a 16 percent increase year over year. 

Regarding the price-to-earnings (P/E) ratio for the same period, the forward P/E ratio for 2024 is approximately 28. This forward P/E reflects market expectations based on projected earnings and current stock price around early 2024.

**Summary:**
- **EPS (Q1 FY 2024):** $2.18  
- **Forward P/E ratio (2024):** About 28

No specific trailing P/E ratio for the exact quarter was provided, but the forward P/E of 28 is the relevant valuation metric for that period.

In [77]:
query = "Lấy Bollinger Bands của cổ phiếu Apple trong 30 ngày gần nhất."
response = agentic_rag.invoke({"question": query})
display(Markdown(response['generation']))

---RETRIEVAL FROM VECTOR DB---


No relevant docs were retrieved using the relevance score threshold 0.3


---CHECK DOCUMENT RELEVANCE TO QUESTION---
---ASSESS GRADED DOCUMENTS---
---DECISION: WEB SEARCH NEEDED, REWRITE QUERY---
---REWRITE QUERY---
---WEB SEARCH---
DOCS TYPE: <class 'list'>
FIRST ELEMENT TYPE: <class 'dict'>
---GENERATE ANSWER---


Để lấy dữ liệu Bollinger Bands của cổ phiếu Apple trong 30 ngày giao dịch gần nhất, bạn cần thực hiện các bước sau:

1. **Thu thập dữ liệu giá cổ phiếu Apple trong 30 ngày giao dịch gần nhất**: Bao gồm giá đóng cửa hàng ngày của cổ phiếu AAPL. Dữ liệu này có thể lấy từ các nguồn dữ liệu tài chính như Yahoo Finance, Google Finance, hoặc các nền tảng giao dịch chứng khoán.

2. **Tính đường trung bình động (Moving Average - MA)**: Thông thường, Bollinger Bands sử dụng đường trung bình động 20 ngày. Bạn tính MA 20 ngày bằng cách lấy trung bình cộng giá đóng cửa của 20 ngày gần nhất.

3. **Tính độ lệch chuẩn (Standard Deviation - SD)**: Tính độ lệch chuẩn của giá đóng cửa trong cùng khoảng thời gian 20 ngày.

4. **Tính dải trên và dải dưới của Bollinger Bands**:
   - Dải trên = MA 20 ngày + (2 x SD)
   - Dải dưới = MA 20 ngày - (2 x SD)

5. **Lặp lại tính toán này cho từng ngày trong 30 ngày giao dịch gần nhất** để có được các giá trị Bollinger Bands tương ứng với từng ngày.

Tóm lại, bạn cần có dữ liệu giá đóng cửa của Apple trong 30 ngày giao dịch gần nhất, sau đó tính đường trung bình động 20 ngày và độ lệch chuẩn 20 ngày để xác định các dải Bollinger Bands. Các dải này sẽ giúp bạn đánh giá mức độ biến động và các điểm đảo chiều tiềm năng của cổ phiếu Apple trong khoảng thời gian đó.

Lưu ý: Trong phần thông tin được cung cấp, chưa có dữ liệu cụ thể về 30 ngày giao dịch gần nhất của Apple, bạn cần truy cập các nguồn dữ liệu tài chính để lấy dữ liệu này và thực hiện tính toán như trên.

In [50]:
query = "what is the capital of India?"
response = agentic_rag.invoke({"question": query})
display(Markdown(response['generation']))

---RETRIEVAL FROM VECTOR DB---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---GRADE: DOCUMENT RELEVANT---
---ASSESS GRADED DOCUMENTS---
---DECISION: DOCUMENTS ARE RELEVANT, GENERATE ANSWER---
---GENERATE ANSWER---


The capital of India is New Delhi. It is a union territory within the megacity of Delhi and is located in the North Indian zone according to traditional Indian geography. New Delhi has a rich history, is home to several monuments, and covers an area of about 42.7 square kilometers. The city has a population of approximately 9.4 million people.

In [65]:
query = "Lấy toàn bộ dữ liệu stock của apple năm 2024 trong database"
response = agentic_rag.invoke({"question": query})
display(Markdown(response['generation']))

---RETRIEVAL FROM VECTOR DB---


No relevant docs were retrieved using the relevance score threshold 0.3


---CHECK DOCUMENT RELEVANCE TO QUESTION---
---NO DOCS, BUT QUESTION SUGGESTS SQL---
---ASSESS GRADED DOCUMENTS---
---DECISION: QUERY SQL DATABASE---
---EXECUTE SQL QUERY OR METRIC COMPUTATION---
Kết nối thành công đến PostgreSQL.


C:\Users\trung\AppData\Local\Temp\ipykernel_14452\3799809131.py:100: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(query, conn)


Lỗi thực thi SQL: Execution failed on sql 'SELECT *
FROM stocks
WHERE date BETWEEN '2024-01-01' AND '2024-12-31'
AND company = 'apple';': column "company" does not exist
LINE 4: AND company = 'apple';
            ^

---GENERATE ANSWER---


Không có dữ liệu về toàn bộ dữ liệu stock của Apple năm 2024 trong cơ sở dữ liệu được cung cấp.

In [34]:
query = "Ai vô địch World Cup 2024?"
response = agentic_rag.invoke({"question": query})
display(Markdown(response['generation']))

---RETRIEVAL FROM VECTOR DB---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT, SUGGEST WEB SEARCH---
---GRADE: DOCUMENT NOT RELEVANT, SUGGEST WEB SEARCH---
---GRADE: DOCUMENT NOT RELEVANT, SUGGEST WEB SEARCH---
---ASSESS GRADED DOCUMENTS---
---DECISION: WEB SEARCH NEEDED, REWRITE QUERY---
---REWRITE QUERY---
---WEB SEARCH---
DOCS TYPE: <class 'list'>
FIRST ELEMENT TYPE: <class 'dict'>
---GENERATE ANSWER---


Dựa trên các thông tin được cung cấp, không có dữ liệu cụ thể hoặc dự đoán rõ ràng về đội tuyển nào sẽ vô địch World Cup 2024. Tuy nhiên, có một số nhận định về các đội bóng mạnh hiện tại:

- Tây Ban Nha được đánh giá rất cao với khả năng tấn công mạnh mẽ và phòng ngự chắc chắn, được dự đoán có cơ hội lớn vô địch EURO 2024 (60,38% theo siêu máy tính Opta) và được dự đoán sẽ thắng Anh trong trận chung kết EURO 2024.
- Pháp là đương kim vô địch World Cup, sở hữu dàn cầu thủ tài năng và kinh nghiệm, với lối chơi tấn công đa dạng và khả năng kiểm soát trận đấu tốt.
- Tuyển Anh cũng được đánh giá cao, có khả năng vô địch EURO 2024 với xác suất 19,9% theo hãng dữ liệu Opta.

Tuy nhiên, các dự đoán này chủ yếu liên quan đến EURO 2024, không phải World Cup 2024. Vì vậy, không có thông tin cụ thể nào về đội tuyển có khả năng vô địch World Cup 2024 trong phần thông tin được cung cấp.

**Tóm lại:** Không có dự đoán rõ ràng về đội vô địch World Cup 2024 trong dữ liệu hiện tại. Các đội tuyển như Tây Ban Nha, Pháp và Anh được xem là những ứng viên mạnh dựa trên phong độ và thành tích gần đây, nhưng chưa có dự đoán chính thức nào cho World Cup 2024.

In [35]:
query = "World Cup gần nhất là đội nào vô địch?"
response = agentic_rag.invoke({"question": query})
display(Markdown(response['generation']))

---RETRIEVAL FROM VECTOR DB---
---CHECK DOCUMENT RELEVANCE TO QUESTION---
---GRADE: DOCUMENT NOT RELEVANT, SUGGEST WEB SEARCH---
---GRADE: DOCUMENT NOT RELEVANT, SUGGEST WEB SEARCH---
---GRADE: DOCUMENT NOT RELEVANT, SUGGEST WEB SEARCH---
---ASSESS GRADED DOCUMENTS---
---DECISION: WEB SEARCH NEEDED, REWRITE QUERY---
---REWRITE QUERY---
---WEB SEARCH---
DOCS TYPE: <class 'list'>
FIRST ELEMENT TYPE: <class 'dict'>
---GENERATE ANSWER---


Đội vô địch World Cup gần đây nhất là đội tuyển Argentina. Trong giải vô địch bóng đá thế giới được tổ chức tại Qatar năm 2022, Argentina đã giành chiến thắng trước Pháp với tỷ số 4–2 trong loạt sút luân lưu sau khi hai đội hòa 3–3 sau hiệp phụ. Đây là lần thứ ba Argentina đoạt cúp vô địch World Cup.

In [36]:
query = "Thủ đô Viêt Nam là gì?"
response = agentic_rag.invoke({"question": query})
display(Markdown(response['generation']))

---RETRIEVAL FROM VECTOR DB---


No relevant docs were retrieved using the relevance score threshold 0.3


---CHECK DOCUMENT RELEVANCE TO QUESTION---
---ASSESS GRADED DOCUMENTS---
---DECISION: WEB SEARCH NEEDED, REWRITE QUERY---
---REWRITE QUERY---
---WEB SEARCH---
DOCS TYPE: <class 'list'>
FIRST ELEMENT TYPE: <class 'dict'>
---GENERATE ANSWER---


Thủ đô của Việt Nam hiện nay là thành phố Hà Nội. Hà Nội là trung tâm chính trị, hành chính quốc gia, nơi đặt trụ sở của các cơ quan trung ương của Đảng, Nhà nước và các tổ chức chính trị - xã hội, cũng như các cơ quan đại diện ngoại giao và tổ chức quốc tế. Đây cũng là trung tâm lớn về văn hóa, giáo dục, khoa học, công nghệ, kinh tế và giao dịch quốc tế của cả nước.

Hà Nội có vị trí ở phía Tây Bắc của vùng đồng bằng châu thổ sông Hồng, với diện tích khoảng 3.359,82 km² và dân số khoảng 8,4 triệu người. Thành phố này đã được UNESCO trao danh hiệu "Thành phố vì hòa bình" vào năm 1999 và được Chủ tịch nước tặng danh hiệu "Thủ đô anh hùng" vào năm 2000. Hà Nội là thủ đô lâu đời nhất trong số 11 thủ đô của các quốc gia Đông Nam Á, với lịch sử hơn 1000 năm.

Theo Hiến pháp năm 2013 của nước Cộng hòa xã hội chủ nghĩa Việt Nam, thủ đô chính thức của Việt Nam là Hà Nội.